# This notebook is used to train and test a model

# Some constants

In [1]:
nrows = 100
ncolumns = 100
batch_size = 50
color_mode = 'grayscale'
input_shape = (nrows, ncolumns, 1)
nclass = 24

# Define Model

In [2]:
from keras.models import Sequential 
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization

def from_paper(input_shape=(100,100,1),nclass=24):
    model = Sequential()
    
    model.add(Conv2D(filters=16,
                     input_shape=input_shape,
                     kernel_size=(3, 3),
                     activation='relu',
                     padding='same'))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(filters=32,
                     kernel_size=(3, 3),
                     activation='relu',
                     padding='same'))
    
    model.add(MaxPooling2D(pool_size=(5, 5)))
    
    model.add(Conv2D(filters=64,
                     kernel_size=(3, 3),
                     activation='relu',
                     padding='same'))
    
    model.add(MaxPooling2D(pool_size=(5, 5)))
    
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    
    model.add(Dense(1024, activation='relu'))
    
    model.add(Dense(nclass, activation='softmax'))
    
    model.summary()
    
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics=['accuracy'])
    
    return model

Using TensorFlow backend.


# Specify the Model

In [3]:
model = from_paper((100,100,1), 24)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 100, 100, 16)      160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 50, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2, 2, 64)         

In [4]:
picture_main_dir = '../dataset/'

print("Data source: " + picture_main_dir)

Data source: ../dataset/


# Specifying Training, Validation and Testing data

In [5]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen =  ImageDataGenerator(rescale=1.0/255, 
                                    rotation_range=15,
                                    zoom_range=0.2)

val_datagen =  ImageDataGenerator(rescale=1.0/255)

test_datagen =  ImageDataGenerator(rescale=1.0/255)

In [6]:
train_gen = train_datagen.flow_from_directory(directory= picture_main_dir + 'train/',
                                              target_size=(nrows, ncolumns),
                                              color_mode=color_mode,
                                              batch_size=batch_size,
                                              class_mode='categorical',
                                              shuffle=False,
                                              seed=0)
val_gen = val_datagen.flow_from_directory(directory= picture_main_dir + 'val/',
                                          target_size=(nrows, ncolumns),
                                          color_mode=color_mode,
                                          batch_size=batch_size,
                                          class_mode='categorical',
                                          shuffle=False,
                                          seed=0)

test_gen = test_datagen.flow_from_directory(directory= picture_main_dir + 'test/',
                                            target_size=(nrows, ncolumns),
                                            color_mode=color_mode,
                                            batch_size=batch_size,
                                            class_mode='categorical',
                                            shuffle=False,
                                            seed=0)

Found 24000 images belonging to 24 classes.
Found 4800 images belonging to 24 classes.
Found 4358 images belonging to 24 classes.


# Training
### Run this section only if you want to train a model, alternatively go to next section and load weights immediately
#### By default this section is skipped

In [7]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
filepath = 'weight file.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', 
                        verbose=1, save_best_only=True, mode='max')

early = EarlyStopping(monitor='val_loss', 
                      mode='min', 
                      patience=40, restore_best_weights=True)

callbacks_list = [checkpoint, early]


In [8]:
# switch to True if you want to train.
if False:
    history = model.fit_generator(train_gen,
                              steps_per_epoch=nclass*1000/batch_size,          
                              validation_data=val_gen,
                              validation_steps=batch_size, 
                              epochs=40, verbose=1,
                              callbacks=callbacks_list)


# Load Weights to the Model

In [9]:
# Load weights
model.load_weights('from_paper_model_weight.h5')

# Test the model

In [10]:
model.evaluate_generator(val_gen, verbose=1)

96/96 [==============================] - 13s 131ms/step


[4.141217232245253e-06, 0.8777083158493042]

In [11]:
model.evaluate_generator(test_gen, verbose=1)

88/88 [==============================] - 14s 163ms/step


[4.172323428974778e-07, 0.8561266660690308]

In [12]:
# model.evaluate_generator(train_gen, verbose=1)

480/480 [==============================] - 121s 251ms/step


[0.00013196583313401788, 0.9981250166893005]

# Letter-wise Prediction

In [21]:
from string import ascii_uppercase
alphabet = [c for c in ascii_uppercase if c not in "JZ"]

## Letter wise for test data

In [22]:
predictions = model.predict_generator(test_gen, verbose=1)

88/88 [==============================] - 4s 45ms/step


In [23]:
import numpy as np

maxAllPredictions = list(map(lambda x: alphabet[np.argmax(x)], predictions))

In [24]:
# Getting correct label for each picture

import os
test_dir=picture_main_dir + 'test/'
letter_dir=test_dir+'{}'

test=[i for i in os.listdir(test_dir)]
test=[f for f in test if not f.endswith('DS_Store')]
testResultArr = []
for letter in test:
    for f in os.listdir(letter_dir.format(letter)):
        testResultArr += letter

In [25]:
import pandas as pd

### Creating the dataframe for each picture

In [26]:
data = {'Predicted': maxAllPredictions,
        'Actual': testResultArr
        }

df = pd.DataFrame(data, columns = ['Predicted', 'Actual'])

In [27]:
output = []
for letter in alphabet:
    cur = df[df.Actual.eq(letter)]
    correct = 0
    totalForLetter = 0
    for index, row in cur.iterrows():
        totalForLetter += 1
        if row['Predicted'] == row['Actual']:
            correct += 1
    output.append([letter, totalForLetter, correct, 1.0*correct/totalForLetter*100])

In [28]:
outputdf = pd.DataFrame(output, columns=['letter', 'no. test data', 'correct predictions', 'accuracy'])
print (outputdf)

   letter  no. test data  correct predictions    accuracy
0       A            162                  115   70.987654
1       B            188                  157   83.510638
2       C            169                  169  100.000000
3       D            203                  193   95.073892
4       E            214                  206   96.261682
5       F            113                  113  100.000000
6       G            205                  205  100.000000
7       H            134                   55   41.044776
8       I            142                  141   99.295775
9       K            132                  130   98.484848
10      L            175                  175  100.000000
11      M            291                   70   24.054983
12      N            171                   65   38.011696
13      O            181                  181  100.000000
14      P            143                  143  100.000000
15      Q            208                  203   97.596154
16      R     